In [5]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from helpers import dataframe

In [6]:
base_url = "https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data"

subjects = ["S01", "S02", "S03"]

fetch_objects = dataframe.generate_fetch_objects(base_url, subjects)

In [7]:
acc_df, hr_df, eda_df, temp_df, bvp_df = dataframe.concatenate_dataframe_from_fetch_objects(fetch_objects)

df = dataframe.combine_dataframe(acc_df, hr_df, eda_df, temp_df, bvp_df)

df = dataframe.fill_missing_values(df)

In [11]:
tags_object = dataframe.generate_tag_objects(base_url, subjects)

timestamps = dataframe.fetch_timestamps_from_tag_objects(tags_object)

df = dataframe.label_dataframe_from_timestamps(df, timestamps)

In [13]:
df.to_csv("data.csv", index=False)